<div class="alert alert-block alert-info">
<center> <h1> Master in Data Science and Advanced Analytics </h1> </center> <br>
<center> PROJECT 2022/2023 </center>

<font size="8">Homework 2: Mini Project - AirBnb Document Database</font>

**Group number: `27`**

**Group Members**:

|      | Student Name  |    Student ID |
|---   |---------------|      ---      |
| 1    | Mahmoud Ahmed |   20221397    |


</div>

# Importing Libraries

In [107]:
import pymongo
# Python Connector

!pip install pymongo
# or #!conda install -y pymongo

from datetime import datetime
from pprint import pprint
import time
from bson.objectid import ObjectId

from pymongo import MongoClient, ASCENDING, DESCENDING

user = "AzureDiamond"
password = "hunter2"
host = "localhost"
port = "27017"
protocol = "mongodb"

client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

# Database check
db = client.sample_airbnb
print(f"Database info: {db}\n")
db.name

Database info: Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'sample_airbnb')



'sample_airbnb'

# Loading and Inspecting database


In [108]:
# Collections are inside our Database 'sample_analytics'
collection_list = db.list_collection_names()

print(f"The database contains {len(collection_list)} collections")
print(f"All collections: {collection_list[0:]}")
print(f"Collection {collection_list[0]} contains {db[collection_list[0]].count_documents({})} documents")

The database contains 6 collections
All collections: ['hosts', 'listings', 'listingsAndReviews_new', 'transactions', 'reviews', 'amenities']
Collection hosts contains 5105 documents


[PyMongo documentation](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html)

# Data modelling

## Apply Schema Versioning Pattern

In [109]:
# Apply the schema versioning pattern to listingsAndReviews collection
db.listingsAndReviews_new.update_many({}, {"$set": {"schema_version": 1}})
db.listingsAndReviews_new.create_index("schema_version")

# Create a new collection with the results of an aggregation operation and apply schema versioning pattern to it.
db.listingsAndReviews_new.aggregate([{"$out": "listings"}])
db.listings.update_many({}, {"$set": {"schema_version": 2}})
db.listings.create_index("schema_version")

'schema_version_1'

## Setup New collections

In [110]:
# Create a new collection to store hosts info
db.drop_collection("hosts")
db.create_collection("hosts")
db.hosts.create_index("schema_version")

# Create a new collection to store reviews list
db.drop_collection("reviews")
db.create_collection("reviews")
db.reviews.create_index("schema_version")

# Create a new collection to store transactions list
db.drop_collection("transactions")
db.create_collection("transactions")
db.transactions.create_index("schema_version")

# Collections are inside our Database 'sample_airbnb'
collection_list = db.list_collection_names()
print(f"The database contains {len(collection_list)} collections: {collection_list[0:]}")

The database contains 6 collections: ['listings', 'reviews', 'hosts', 'listingsAndReviews_new', 'transactions', 'amenities']


## Hosts

Extract hosts info from listings collection and store it in hosts collection.

In [111]:
# Create index on host_id field to speed up the query below.
db.listings.create_index("host_id")

'host_id_1'

In [112]:
# Extract hosts info from listings collection and store it in hosts collection (version 2) using aggregation pipeline.
setup_hosts_pipeline_1 = [
    {
        "$project": {  # project only the fields we need
            "_id": "$host_id",
            "hostFields": {
                "$filter": {
                    "input": {"$objectToArray": "$$ROOT"},  # convert document to array of key-value pairs
                    "as": "field",
                    "cond": {"$regexMatch": {"input": "$$field.k", "regex": ".*host.*"}}
                    # filter only fields starting with 'host'
                }
            }
        }
    },
    {
        "$addFields": {
            "schema_version": 2,  # add schema version
            "hostInfo": {
                "$arrayToObject": {  # convert array of key-value pairs back to document
                    "$map": {
                        "input": "$hostFields",  # use the filtered array of key-value pairs
                        "as": "field",
                        "in": {"k": "$$field.k", "v": "$$field.v"}
                    }
                }
            }
        }
    },
    {
        "$project": {
            "hostFields": 0  # remove the filtered array of key-value pairs
        }
    },
    {
        "$merge": {  # merge the new document with the existing one
            "into": "hosts",
            "on": "_id",
            "whenMatched": "replace",  # replace the existing document with the new one
            "whenNotMatched": "insert"
        }
    }
]
db.listings.aggregate(setup_hosts_pipeline_1)
db.hosts.find_one()

{'_id': '51399391',
 'schema_version': 2,
 'hostInfo': {'host_about': 'Gostamos de passear, de viajar, de conhecer pessoas e locais novos, gostamos de desporto e animais! Vivemos na cidade mais linda do mundo!!!',
  'host_has_profile_pic': True,
  'host_id': '51399391',
  'host_identity_verified': True,
  'host_is_superhost': False,
  'host_listings_count': 3,
  'host_location': 'Porto, Porto District, Portugal',
  'host_name': 'Ana&Gonçalo',
  'host_neighbourhood': '',
  'host_picture_url': 'https://a0.muscache.com/im/pictures/fab79f25-2e10-4f0f-9711-663cb69dc7d8.jpg?aki_policy=profile_x_medium',
  'host_response_rate': 100,
  'host_response_time': 'within an hour',
  'host_thumbnail_url': 'https://a0.muscache.com/im/pictures/fab79f25-2e10-4f0f-9711-663cb69dc7d8.jpg?aki_policy=profile_small',
  'host_total_listings_count': 3,
  'host_url': 'https://www.airbnb.com/users/show/51399391',
  'host_verifications': ['email',
   'phone',
   'reviews',
   'jumio',
   'offline_government_id',
 

In [113]:
# Extract hosts info from listings collection and store it in hosts collection (version 2) using aggregation pipeline.
setup_hosts_pipeline_2 = [
    {
        "$replaceWith": {  # replace the existing document with the new one
            "$mergeObjects": [
                {"schema_version": 2},  # add schema version
                "$hostInfo",  # add all fields from hostInfo document to the existing document
                {"_id": "$hostInfo.host_id"}  # add _id field with the value of host_id field
            ]
        }
    }, {
        "$out": "hosts"
    }
]
db.hosts.aggregate(setup_hosts_pipeline_2)
db.hosts.update_many({}, {"$unset": {"host_id": ""}})
db.hosts.find_one()

{'_id': '51399391',
 'schema_version': 2,
 'host_about': 'Gostamos de passear, de viajar, de conhecer pessoas e locais novos, gostamos de desporto e animais! Vivemos na cidade mais linda do mundo!!!',
 'host_has_profile_pic': True,
 'host_identity_verified': True,
 'host_is_superhost': False,
 'host_listings_count': 3,
 'host_location': 'Porto, Porto District, Portugal',
 'host_name': 'Ana&Gonçalo',
 'host_neighbourhood': '',
 'host_picture_url': 'https://a0.muscache.com/im/pictures/fab79f25-2e10-4f0f-9711-663cb69dc7d8.jpg?aki_policy=profile_x_medium',
 'host_response_rate': 100,
 'host_response_time': 'within an hour',
 'host_thumbnail_url': 'https://a0.muscache.com/im/pictures/fab79f25-2e10-4f0f-9711-663cb69dc7d8.jpg?aki_policy=profile_small',
 'host_total_listings_count': 3,
 'host_url': 'https://www.airbnb.com/users/show/51399391',
 'host_verifications': ['email',
  'phone',
  'reviews',
  'jumio',
  'offline_government_id',
  'government_id']}

Remove host info from listings collection to reduce its size.

In [114]:
setup_hosts_pipeline_3 = {
    "$unset": {
        "hostInfo": "",
        "host_about": "",
        "host_has_profile_pic": "",
        "host_identity_verified": "",
        "host_is_superhost": "",
        "host_listings_count": "",
        "host_location": "",
        "host_name": "",
        "host_neighbourhood": "",
        "host_picture_url": "",
        "host_response_rate": "",
        "host_response_time": "",
        "host_thumbnail_url": "",
        "host_total_listings_count": "",
        "host_url": "",
        "host_verifications": ""
    }
}
db.listings.update_many({}, setup_hosts_pipeline_3)

## Reviews

Extract reviews info from listings collection and store it in reviews collection.

In [115]:
# Create index on listing_id field to enable faster queries on reviews collection.
db.reviews.create_index("listing_id")

'listing_id_1'

Merge intersection of reviews from listings into a new collection.

In [116]:
setup_reviews_pipeline = [
    {
        "$project": {
            "reviews": {
                "$setIntersection": ["$reviews", "$reviews_copy1", "$reviews_copy2", "$reviews_copy3", "$reviews_copy4"]
            }
        }
    },
    {
        "$unwind": "$reviews"
    },
    {
        "$replaceRoot": {
            "newRoot": {
                "$mergeObjects": [
                    {"schema_version": 2},
                    "$reviews",
                ]
            }
        }
    },
    {
        "$out": "reviews"
    }
]
db.listings.aggregate(setup_reviews_pipeline)
db.reviews.find_one()

{'_id': '108444999',
 'schema_version': 2,
 'date': datetime.datetime(2016, 10, 16, 4, 0),
 'listing_id': '10006546',
 'reviewer_id': '24960918',
 'reviewer_name': 'Hanneke',
 'comments': 'The appartment was great, it is really nice and cosy, and it is situated at a perfect location. Ana and Goncalo were very friendly, they were very helpfull during our stay. We had a great time during our stay in Porto!'}

Fetch reviews from reviews collection and add latest 5 reviews to listings collection.

In [117]:
reviews_lookup_stage = [{
    "$lookup": {
        "from": "reviews",
        "localField": "_id",
        "foreignField": "listing_id",
        "as": "LATEST_REVIEWS"
    }}, {"$sort": {"LATEST_REVIEWS.date": -1}}, {
    "$addFields": {
        "LATEST_REVIEWS": {
            "$slice": ["$LATEST_REVIEWS", 5],
        }
    }}, {
    "$out": "listings"
}
]

db.listings.aggregate(reviews_lookup_stage)

In [118]:
db.listings.find_one()

{'_id': '16100663',
 'listing_url': 'https://www.airbnb.com/rooms/16100663',
 'name': 'Styled Apartment in the heart of HK.- Wanchai',
 'summary': 'This charming one bedroom apartment is centrally located in the Star Street neighborhood in the heart of Wanchai. Live like a local while surrounded by the best restaurants and shopping the city has to offer.  Enjoy the convenient access to all subway lines, the best shopping for boutique and high end fashion is around the corner as well as the CBD and lot of tourist spots .',
 'space': "Star Street is surrounded by beautiful architecture and tons of high end boutiques for fashion, design, furniture and the trendy pop-up stores. There are many amazing restaurants in the area and other cocktail bars however our street is extremely quiet for a good night's sleep. Our place is good for couples, solo adventurers, and business travelers that don't care for plain hotels.  Enjoy Hong Kong from our bright and charming one-bedroom located in a tradi

## Transactions

Extract transactions info from listings collection and store it in transactions collection.

In [119]:
# Create index on listing_id field to enable faster queries on transactions collection.
db.transactions.create_index("listing_id")

'listing_id_1'

In [120]:
setup_transactions_pipeline_1 = [
    {
        "$project": {
            "listing_id": "$_id",
            "transactions": 1
        }
    },
    {
        "$replaceRoot": {
            "newRoot": {
                "$mergeObjects": [
                    {"schema_version": 2},  # add schema version
                    "$transactions",
                    {"listing_id": "$listing_id"}
                ]
            }
        }
    },
    {
        "$out": "transactions"
    }
]
db.listings.aggregate(setup_transactions_pipeline_1)

# Remove transactions field from listings collection to remove redundant data.
db.listings.update_many({}, {"$unset": {"transactions": ""}})
db.transactions.find_one()

{'_id': ObjectId('64450bd9926abc8f513a5076'),
 'schema_version': 2,
 'bucket_end_date': datetime.datetime(2017, 1, 5, 0, 0),
 'bucket_start_date': datetime.datetime(1964, 5, 14, 0, 0),
 'transaction_count': 69,
 'transactions': [{'date': datetime.datetime(2009, 3, 10, 0, 0),
   'price': '8.209489528793955770424872753210365772247314453125'},
  {'date': datetime.datetime(2006, 4, 17, 0, 0),
   'price': '18.437264475003619423887357697822153568267822265625'},
  {'date': datetime.datetime(1982, 11, 19, 0, 0),
   'price': '17.296076700616453081238432787358760833740234375'},
  {'date': datetime.datetime(2016, 10, 19, 0, 0),
   'price': '144.17453082689581833619740791618824005126953125'},
  {'date': datetime.datetime(2010, 9, 7, 0, 0),
   'price': '105.9250892743451544220079085789620876312255859375'},
  {'date': datetime.datetime(2012, 6, 25, 0, 0),
   'price': '9.456160047055874429133837111294269561767578125'},
  {'date': datetime.datetime(2016, 11, 9, 0, 0),
   'price': '70.32362333531598608

# Questions

### Question 1

The most typical use case for the database is to show information relating to a property listing to a customer. This is done by a query to the database which returns one of the listing documents. Currently a lot of time is spent when a listing is retrieved from the database to show to a customer. Decide what information should be returned in a typical query and optimise the structure for this use case. For example, we typically only want a sample of reviews but not all reviews (although all reviews ), the customer does not need to know past transaction data, etc. Update the document schema for this typical use case. This may involve the creation of new collections and documents.

In [121]:
# Sample listing id
listing_sample_id = "10006546"

# Match stage to filter listings based on _id
match_stage = {
    "$match": {
        "_id": listing_sample_id
    }
}

# Project stage to select only the fields we need
project_stage = {
    "$project": {
        "_id": 1,
        "name": 1,
        "summary": 1,
        "neighborhood_overview": 1,
        "transit": 1,
        "access": 1,
        "house_rules": 1,
        "listing_url": 1,
        "images.picture_url": 1,
        "neighbourhood": 1,
        "address.street": 1,
        "address.market": 1,
        "address.country_code": 1,
        "address.country": 1,
        "property_type": 1,
        "room_type": 1,
        "accommodates": 1,
        "bathrooms": 1,
        "bedrooms": 1,
        "beds": 1,
        "bed_type": 1,
        "amenities": 1,
        "price": 1,
        "security_deposit": 1,
        "cleaning_fee": 1,
        "minimum_nights": 1,
        "maximum_nights": 1,
        "availability.availability_30": 1,
        "availability.availability_60": 1,
        "availability.availability_90": 1,
        "number_of_reviews": 1,
        "host_id": 1,
        "LATEST_REVIEWS": 1,
    }
}

Fetch host details from hosts collection and add it to listings collection using host_id.

In [122]:
hosts_lookup_stage = [{
    "$lookup": {
        "from": "hosts",
        "localField": "host_id",
        "foreignField": "_id",
        "as": "hostDetails"
    }}, {"$unwind": "$hostDetails"}, {
    "$addFields": {
        "hostDetails": {
            "_id": 1,
            "host_name": 1,
            "host_since": 1,
            "host_about": 1,
            "host_location": 1,
            "host_identity_verified": 1,
            "host_is_superhost": 1,
            "host_listings_count": 1,
            "host_verifications": 1,
            "host_response_time": 1,
            "host_response_rate": 1,
            "host_acceptance_rate": 1,
            "host_thumbnail_url": 1,
            "host_total_listings_count": 1,
        }
    }
}]

In [123]:
# Define the pipeline for the aggregation query
pipeline = [
    match_stage,
    project_stage,
    *hosts_lookup_stage,
]

# Execute the aggregation query
result1 = list(db.listings.aggregate(pipeline))
pprint(result1)

[{'LATEST_REVIEWS': [{'_id': '108444999',
                      'comments': 'The appartment was great, it is really nice '
                                  'and cosy, and it is situated at a perfect '
                                  'location. Ana and Goncalo were very '
                                  'friendly, they were very helpfull during '
                                  'our stay. We had a great time during our '
                                  'stay in Porto!',
                      'date': datetime.datetime(2016, 10, 16, 4, 0),
                      'listing_id': '10006546',
                      'reviewer_id': '24960918',
                      'reviewer_name': 'Hanneke',
                      'schema_version': 2},
                     {'_id': '111065376',
                      'comments': 'Kävimme majoittajan kanssa viestinvaihtoa '
                                  'ennen lomaa ja he antoivat mukavan '
                                  'vaikutelman. He olivatkin hyv

### Question 2
Review the data for any errors (such as transactions that don’t fit the listing) or inappropriate duplication and clean up as appropriate.

Done on steps above by :
- Migrating hosts, reviews and transactions data to separate collections.
- Removing redundant and duplicated data from listings collection.

Remove reviews arrays and its copies from listings collection to remove redundant data.

In [124]:
db.listings.update_many({}, {
    "$unset": {"reviews": "", "reviews_copy1": "", "reviews_copy2": "", "reviews_copy3": "", "reviews_copy4": ""}
})
db.listings.find_one()

{'_id': '16100663',
 'listing_url': 'https://www.airbnb.com/rooms/16100663',
 'name': 'Styled Apartment in the heart of HK.- Wanchai',
 'summary': 'This charming one bedroom apartment is centrally located in the Star Street neighborhood in the heart of Wanchai. Live like a local while surrounded by the best restaurants and shopping the city has to offer.  Enjoy the convenient access to all subway lines, the best shopping for boutique and high end fashion is around the corner as well as the CBD and lot of tourist spots .',
 'space': "Star Street is surrounded by beautiful architecture and tons of high end boutiques for fashion, design, furniture and the trendy pop-up stores. There are many amazing restaurants in the area and other cocktail bars however our street is extremely quiet for a good night's sleep. Our place is good for couples, solo adventurers, and business travelers that don't care for plain hotels.  Enjoy Hong Kong from our bright and charming one-bedroom located in a tradi

### Question 3
Once per month we like to reward hosts with recognition. Pick three superhosts who have at least two property listings that can accommodate more than four people?

#### Database Optimization

Create indexes on the fields that are used in the aggregation query to improve the performance of the query.

In [125]:
db.hosts.create_index("host_is_superhost")
db.hosts.create_index("host_listings_count")
db.hosts.create_index([("host_is_superhost", DESCENDING),
                       ("host_listings_count", DESCENDING)])

'host_is_superhost_-1_host_listings_count_-1'

#### Data Query

Write an aggregation query to fetch the top 3 superhosts who have at least two property listings that can accommodate more than four people.

In [126]:
# Match stage to filter listings based on host_is_superhost and host_listings_count
match_stage3 = {
    "$match": {
        "host_is_superhost": True,
        "host_listings_count": {"$gte": 2}
    }
}

# Lookup stage to fetch listings from another collection (assuming host_id is a field in the listing document)
listings_lookup_stage3 = [{
    "$lookup": {
        "from": "listings",
        "localField": "_id",
        "foreignField": "host_id",
        "as": "listings"
    }}, {
    "$match": {
        "listings.accommodates": {"$gt": 4},
    }}, {
    "$addFields": {
        "totalListings": {"$size": "$listings"}
    }
}]

# Sort stage to sort the listings based on totalListings in descending order and limit to top 3
sort_stage3 = [{
    "$sort": {
        "totalListings": -1
    }
}, {"$limit": 3}]

# Project stage to select only the fields we need
project_stage3 = {
    "$project": {
        "_id": 1,
        "host_name": 1,
        "totalListings": 1,
    }
}

# Define the pipeline for the aggregation query
pipeline3 = [
    match_stage3,
    *listings_lookup_stage3,
    *sort_stage3,
    project_stage3
]

# Execute the aggregation query
result3 = list(db.hosts.aggregate(pipeline3))
pprint(result3)

[{'_id': '30907275', 'host_name': 'Liiiving', 'totalListings': 7},
 {'_id': '16922131', 'host_name': 'YourOpo', 'totalListings': 5},
 {'_id': '10496350', 'host_name': 'Elite', 'totalListings': 4}]


### Question 4

One of employees is thinking of buying a property to rent out. Which bed type is most common in the listings that have waterfront and a dishwasher in New York?

#### Database Optimization

Create indexes on the fields that are used in the aggregation query to improve the performance of the query.

In [127]:
db.listings.create_index([("address.market", ASCENDING)])
db.listings.create_index([("amenities", DESCENDING)])

'amenities_-1'

#### Data Query

Write an aggregation query to fetch the bed type that is most common in the listings that have waterfront and a dishwasher in New York.

In [128]:
# Match stage to filter listings based on address.market and amenities
match_stage4 = {
    "$match": {
        "address.market": "New York",
        "amenities": {"$all": ["Waterfront", "Dishwasher"]}
    }
}

# Group stage to group the listings based on bed_type and count the number of listings in each group
group_stage4 = [{
    "$group": {
        "_id": "$bed_type",
        "count": {"$sum": 1}
    }
}, {"$limit": 1}]

# Sort stage to sort the listings based on count in descending order and limit to top 1
sort_stage4 = [
    {
        "$sort": {"count": -1}
    },
    {"$limit": 1}
]

# Define the pipeline for the aggregation query
pipeline4 = [
    match_stage4,
    *group_stage4,
]

# Execute the aggregation query
result4 = list(db.listings.aggregate(pipeline4))
pprint(result4)

[{'_id': 'Real Bed', 'count': 1}]


### Question 5

We are thinking of identifying someone to hire to write review for us professionally. What is the name of the reviewer who left the longest review in New York?

#### Database Optimization

Create indexes on the fields that are used in the aggregation query to improve the performance of the query.

In [129]:
# Create index on reviews fields reviewer_id, reviewer_name, comments
db.reviews.create_index("reviewer_id")
db.reviews.create_index("reviewer_name")
db.reviews.create_index([("reviewer_id", DESCENDING), ("comments", DESCENDING)])
# listing_id already indexed

'reviewer_id_-1_comments_-1'

#### Data Query

Write an aggregation query to fetch the name of the reviewer who left the longest review in New York.

In [130]:
# Match stage to filter reviews based on listing.address.market
listings_lookup_stage5 = [{
    "$lookup": {
        "from": "listings",
        "localField": "listing_id",
        "foreignField": "_id",
        "as": "listing"
    }}, {
    "$unwind": "$listing"
}]

# Match stage to filter reviews based on listing.address.market
match_stage5 = {
    "$match": {
        "listing.address.market": "New York"
    }
}

# Sort stage to sort the reviews based on comments length in descending order and limit to top 1
sort_stage5 = [{
    "$sort": {
        "comments.length": -1
    }
}, {"$limit": 1}]

# Project stage to select only the fields we need
project_stage5 = {
    "$project": {
        "_id": 0,
        "reviewer_name": 1,
        "reviewer_id": 1,
        "comments": 1,
    }
}

# Define the pipeline for the aggregation query
pipeline5 = [
    *listings_lookup_stage5,
    match_stage5,
    *sort_stage5,
    project_stage5
]

# Execute the aggregation query
result5 = list(db.reviews.aggregate(pipeline5))
pprint(result5)

[{'comments': 'Josh was out of town during my 1 month stay. His roommates '
              'greeted and helped get me settled. They were great hosts and '
              "all around cool people. I'm a Brooklynite, but have never lived "
              'in Bushwick.\r\n'
              "If you're looking for an hip, authentic, and convenient "
              'Brooklyn experience, this spot is for you.  You can literally '
              "see the Subway platform from Josh's window. Also a couple steps "
              'away from anything you could possibly need... restaurants, '
              'juice bar, organic grocery, etc. ',
  'reviewer_id': '52006105',
  'reviewer_name': 'Antoine'}]


### Question 6

We are thinking of trying to assess the security of different areas based on the security deposit required. What is the biggest and smallest difference between the price and deposit for security per number of visitors staying at the property?

Project stage to select only the fields we need and calculate the difference between price and security_deposit for each listing and store it in a new field price_deposit_diff then calculate the max and min price_deposit_diff for each group.

#### Database Optimization

Create indexes on the fields that are used in the aggregation query to improve the performance of the query.

In [131]:
db.listings.create_index("price")
db.listings.create_index([("price", DESCENDING)])
db.listings.create_index("security_deposit")
db.listings.create_index([("security_deposit", DESCENDING)])
db.listings.create_index("accommodates")
db.listings.create_index([("accommodates", DESCENDING)])
db.listings.create_index([("address.market", DESCENDING), ("price", ASCENDING), ("security_deposit", ASCENDING),
                          ("accommodates", DESCENDING)])

'address.market_-1_price_1_security_deposit_1_accommodates_-1'

#### Data Query

Select only the fields we need and calculate the difference between price and security_deposit for each listing and store it in a new field price_deposit_diff then calculate the max and min price_deposit_diff for each group.

In [132]:
project_stage6 = {
    "$project": {
        "_id": 0,
        "address.market": 1,
        "price": 1,
        "security_deposit": 1,
        "accommodates": 1,
        "price_deposit_diff": {"$subtract": ["$price", "$security_deposit"]}
    }
}
group_stage6 = {
    "$group": {
        "_id": "$address.market",  # group by government_area | group by market based on business requirement
        "max_diff": {"$max": "$price_deposit_diff"},
        "min_diff": {"$min": "$price_deposit_diff"}
    }
}

In [133]:
# Define the pipeline for the aggregation query
pipeline6 = [
    project_stage6,
    group_stage6
]

# Execute the aggregation query
result6 = list(db.listings.aggregate(pipeline6))
pprint(result6)

[{'_id': 'Montreal',
  'max_diff': Decimal128('1200.00'),
  'min_diff': Decimal128('-6419.00')},
 {'_id': 'Other (International)',
  'max_diff': Decimal128('420.00'),
  'min_diff': Decimal128('392.00')},
 {'_id': 'Oahu',
  'max_diff': Decimal128('1500.00'),
  'min_diff': Decimal128('-2210.00')},
 {'_id': '',
  'max_diff': Decimal128('-1.00'),
  'min_diff': Decimal128('-1176.00')},
 {'_id': 'Kauai',
  'max_diff': Decimal128('536.00'),
  'min_diff': Decimal128('-4000.00')},
 {'_id': 'Maui',
  'max_diff': Decimal128('692.00'),
  'min_diff': Decimal128('-1200.00')},
 {'_id': 'Barcelona',
  'max_diff': Decimal128('2700.00'),
  'min_diff': Decimal128('-4373.00')},
 {'_id': 'Porto',
  'max_diff': Decimal128('450.00'),
  'min_diff': Decimal128('-975.00')},
 {'_id': 'The Big Island',
  'max_diff': Decimal128('2250.00'),
  'min_diff': Decimal128('-1400.00')},
 {'_id': 'New York',
  'max_diff': Decimal128('300.00'),
  'min_diff': Decimal128('-4815.00')},
 {'_id': 'Istanbul',
  'max_diff': Decimal

### Question 7

We want to identify areas by whether they are typically used for short breaks, like weekend mini breaks, or whether they are more suitable for long trips. We will use this information to target advertising of different customers. It is not expected this information will change much over time so we won’t look to update it we just would like a current view. What is the average duration of stay (in nights) per type of property per city (you can use the maximum_nights to measure length of stays)? For each property type return the city with the highest and lowest average value.

#### Database Optimization

Create indexes on the fields that are used in the aggregation query to improve the performance of the query.

In [134]:
db.listings.create_index("property_type")
db.listings.create_index([("maximum_nights", DESCENDING)])
db.listings.create_index([("minimum_nights", ASCENDING)])
db.listings.create_index([("maximum_nights", DESCENDING), ("minimum_nights", ASCENDING)])

'maximum_nights_-1_minimum_nights_1'

Set the field type of maximum_nights and minimum_nights to integer

In [135]:
db.listings.update_many({}, [
    {
        "$set": {
            "maximum_nights": {"$toInt": "$maximum_nights"},
            "minimum_nights": {"$toInt": "$minimum_nights"}
        }
    }
])

#### Data Query

Group stage to group the listings based on address.market (city | area) and property_type and calculate the average maximum_nights then group the listings based on property_type and calculate the city with highest and lowest average maximum_nights for each group

In [136]:
group_stage7 = {
    "$group": {
        "_id": {"city": "$address.market", "property_type": "$property_type"},
        "avgDuration": {"$avg": "$maximum_nights"}
    }
}

group_stage7_1 = {
    "$group": {
        "_id": "$_id.property_type",
        "cityWithHighestAvg": {"$max": {"city": "$_id.city", "avgDuration": "$avgDuration"}},
        "cityWithLowestAvg": {"$min": {"city": "$_id.city", "avgDuration": "$avgDuration"}}
    }
}

Project stage to select only the fields we need and rename the _id field to propertyType

In [137]:
project_stage7 = {
    "$project": {
        "_id": 0,
        "propertyType": "$_id",
        "cityWithHighestAvg": 1,
        "cityWithLowestAvg": 1
    }
}

# Define the pipeline for the aggregation query
pipeline7 = [
    group_stage7,
    group_stage7_1,
    project_stage7
]

# Execute the aggregation query
result7 = list(db.listings.aggregate(pipeline7))
pprint(result7)

[{'cityWithHighestAvg': {'avgDuration': 1125.0, 'city': 'Sydney'},
  'cityWithLowestAvg': {'avgDuration': 1125.0, 'city': 'Sydney'},
  'propertyType': 'Train'},
 {'cityWithHighestAvg': {'avgDuration': 906.0, 'city': 'The Big Island'},
  'cityWithLowestAvg': {'avgDuration': 120.0, 'city': 'Maui'},
  'propertyType': 'Cabin'},
 {'cityWithHighestAvg': {'avgDuration': 1125.0, 'city': 'Montreal'},
  'cityWithLowestAvg': {'avgDuration': 1125.0, 'city': 'Montreal'},
  'propertyType': 'Castle'},
 {'cityWithHighestAvg': {'avgDuration': 1125.0, 'city': 'The Big Island'},
  'cityWithLowestAvg': {'avgDuration': 531.625, 'city': 'Barcelona'},
  'propertyType': 'Bed and breakfast'},
 {'cityWithHighestAvg': {'avgDuration': 445.0, 'city': 'The Big Island'},
  'cityWithLowestAvg': {'avgDuration': 15.0, 'city': 'Barcelona'},
  'propertyType': 'Villa'},
 {'cityWithHighestAvg': {'avgDuration': 419.5, 'city': 'The Big Island'},
  'cityWithLowestAvg': {'avgDuration': 1125.0, 'city': 'Barcelona'},
  'property

### Question 8

We want to have a new webpage for hosts when setting up their account. It will list suggested typical amenities. This data will need to be available every time a host registers a property but is not expected to change very much. The starting point for the list will be all unique amenities currently listed in properties (across all documents). Optimise the database for this use case and show how the data should be queried.

#### Database Optimisation

Create a new collection called amenities and add an index on the name field. The name field should be unique and the schema_version field should be indexed. The schema_version field should be added to the collection with a default value of 2. The amenities field is already indexed in the listings collection.

In [138]:
db.amenities.drop()
db.create_collection("amenities")
db.amenities.create_index("schema_version")

db.amenities.create_index("name", unique=True)

'name_1'

In [139]:
# find documents with empty string in the 'name' field and delete them.
result8_2 = db.amenities.delete_many({"name": ""})
print("Number of documents deleted: ", result8_2.deleted_count)

Number of documents deleted:  0


In [140]:
setup_amenities_pipeline8 = [
    {
        "$unwind": "$amenities"
    },
    {
        "$group": {
            "_id": "$amenities"
        }
    },
    {
        "$addFields": {
            "_id": None,
            "name": "$_id",
            "schema_version": 2
        }
    }
]
# Execute the aggregation pipeline and fetch the results
result8 = list(db.listings.aggregate(setup_amenities_pipeline8))
for doc in result8:
    doc["_id"] = ObjectId()  # Set _id to ObjectId
db.amenities.insert_many(result8)
db.amenities.find_one()

{'_id': ObjectId('64450c1bf0aa1711e391e56a'),
 'name': 'Bed linens',
 'schema_version': 2}

#### Data Query

Query the amenities collection to return the first 10 amenities

In [141]:
result8_1 = list(db.amenities.find({}, {"_id": 0, "name": 1}).limit(10))
top_amenities = [amenity['name'] for amenity in result8_1]
pprint(top_amenities)

['Bed linens',
 'Heated towel rack',
 'Sauna',
 'Host greets you',
 'Bathtub',
 'Sun loungers',
 'Pack ’n Play/travel crib',
 'Wifi',
 'Children’s dinnerware',
 'Garden or backyard']


### Question 9

We want to start tracking our reviewers better. We want to create a webpage that shows the top 20 reviewers and the count of the number of reviews of each of these reviewers. This webpage should be kept up to date. It should also have a link to return the number of reviews for a given reviewer ID or Name (show how to query for number of reviews by ID or query quickly).

In [142]:
# Define the pipeline for the aggregation query
group_stage9 = {
    "$group": {
        "_id": "$reviewer_id",
        "review_count": {"$sum": 1},
        "reviewer_name": {"$first": "$reviewer_name"}
    }
}

project_stage9 = {
    "$project": {
        "_id": 0,
        "reviewer_id": "$_id",
        "reviewer_name": 1,
        "review_count": 1
    }
}

sort_stage9 = {
    "$sort": {"review_count": -1}
}

limit_stage9 = {
    "$limit": 20
}

pipeline9 = [
    group_stage9,
    project_stage9,
    sort_stage9,
    limit_stage9
]

# Execute the aggregation query
result9 = list(db.reviews.aggregate(pipeline9))
pprint(result9)

[{'review_count': 24, 'reviewer_id': '20775242', 'reviewer_name': 'Filipe'},
 {'review_count': 13, 'reviewer_id': '67084875', 'reviewer_name': 'Nick'},
 {'review_count': 10, 'reviewer_id': '2961855', 'reviewer_name': 'Uge'},
 {'review_count': 9, 'reviewer_id': '20991911', 'reviewer_name': 'Lisa'},
 {'review_count': 9, 'reviewer_id': '162027327', 'reviewer_name': 'Thien'},
 {'review_count': 8, 'reviewer_id': '60816198', 'reviewer_name': 'Todd'},
 {'review_count': 8, 'reviewer_id': '69140895', 'reviewer_name': 'Lisa'},
 {'review_count': 8, 'reviewer_id': '12679057', 'reviewer_name': 'Jodi'},
 {'review_count': 8, 'reviewer_id': '1705870', 'reviewer_name': 'David'},
 {'review_count': 8, 'reviewer_id': '55241576', 'reviewer_name': 'Courtney'},
 {'review_count': 7, 'reviewer_id': '47303133', 'reviewer_name': 'Lance'},
 {'review_count': 7, 'reviewer_id': '78093968', 'reviewer_name': 'David'},
 {'review_count': 6, 'reviewer_id': '57325457', 'reviewer_name': 'Mary'},
 {'review_count': 6, 'revie

In [143]:
# Query the number of reviews for a given reviewer ID
result9_1 = db.reviews.count_documents({"reviewer_id": result9[0]["reviewer_id"]})
result9_1
# Querying by name gives different results as we have multiple reviewers with the same name.

24

### Question 10

For each property we store review scores across different metrics (accuracy, check-in, cleanliness etc). We are thinking we may soon add more metrics, although we don’t know what these will be. We want to be able to easily query the average score across all of these metrics, including any new metrics that might be added without changing the query. Adjust the data model so this can be done and show the query for an example property.

#### Database Optimisation

Create a new field called REVIEWS_SCORES in the listings collection. The REVIEWS_SCORES field should be an array of embedded documents. Each embedded document should have a metric field and a score field. The metric field should be the name of the review score metric and the score field should be the value of the review score metric. The REVIEWS_SCORES field should be indexed.

In [144]:
# Define the index on the "metric" and "score" fields within the "REVIEWS_SCORES" document
db.listings.create_index("REVIEWS_SCORES")

'REVIEWS_SCORES_1'

In [145]:
setup_reviews_scores_pipeline10_1 = [{
    "$addFields": {
        "REVIEWS_SCORES": {
            "checkin": "$review_scores_checkin",
            "cleanliness": "$review_scores_cleanliness",
            "communication": "$review_scores_communication",
            "location": "$review_scores_location",
            "rating": "$review_scores_rating",
            "value": "$review_scores_value",
        }
    }
}, {"$out": "listings"}]
db.listings.aggregate(setup_reviews_scores_pipeline10_1)
db.listings.find_one({}, {"REVIEWS_SCORES": 1})

{'_id': '16100663',
 'REVIEWS_SCORES': {'checkin': 10,
  'cleanliness': 10,
  'communication': 10,
  'location': 10,
  'rating': 96,
  'value': 9}}

Updating listings collection by removing reviews scores fields.

In [146]:
setup_reviews_scores_pipeline10_2 = {
    "$unset": {
        "review_scores_checkin": "",
        "review_scores_cleanliness": "",
        "review_scores_communication": "",
        "review_scores_location": "",
        "review_scores_rating": "",
        "review_scores_value": ""
    }
}
db.listings.update_many({}, setup_reviews_scores_pipeline10_2)

#### Data Query

Query the listings collection to return the first document with the REVIEWS_SCORES field.

In [147]:
db.listings.find_one({}, {"REVIEWS_SCORES": 1})

{'_id': '16100663',
 'REVIEWS_SCORES': {'checkin': 10,
  'cleanliness': 10,
  'communication': 10,
  'location': 10,
  'rating': 96,
  'value': 9}}

### Question 11

We wish to be able to have better access to information about transaction, we wish to develop a search engine that can calculate the average value of transactions in a given period of time quickly for a given property.

#### Database Optimisation

Transactions collection has been created and data extracted from the listings collection in early step. The transactions collection has a field called price which is a string. We wish to convert this field to a decimal type. We also wish to round the price to 4 decimal places.

In [148]:
# Define the index on the "date" and "price" fields within the "transactions" array
db.transactions.create_index("transactions")
index_definition = [
    ("transactions.date", DESCENDING),
    ("transactions.price", DESCENDING)
]

# Create the index
db.transactions.create_index(index_definition)

'transactions.date_-1_transactions.price_-1'

In [149]:
# Convert price field from string to decimal.
setup_transactions_pipeline_11 = [
    {
        "$set": {
            "transactions": {
                "$map": {
                    "input": "$transactions",
                    "as": "transaction",
                    "in": {
                        "date": "$$transaction.date",
                        "price": {"$round": [{"$toDecimal": "$$transaction.price"}, 4]},
                    }
                }
            }
        }
    }
]

db.transactions.update_many({}, setup_transactions_pipeline_11)
db.transactions.find_one()

{'_id': ObjectId('64450bd9926abc8f513a5076'),
 'schema_version': 2,
 'bucket_end_date': datetime.datetime(2017, 1, 5, 0, 0),
 'bucket_start_date': datetime.datetime(1964, 5, 14, 0, 0),
 'transaction_count': 69,
 'transactions': [{'date': datetime.datetime(2009, 3, 10, 0, 0),
   'price': Decimal128('8.2095')},
  {'date': datetime.datetime(2006, 4, 17, 0, 0),
   'price': Decimal128('18.4373')},
  {'date': datetime.datetime(1982, 11, 19, 0, 0),
   'price': Decimal128('17.2961')},
  {'date': datetime.datetime(2016, 10, 19, 0, 0),
   'price': Decimal128('144.1745')},
  {'date': datetime.datetime(2010, 9, 7, 0, 0),
   'price': Decimal128('105.9251')},
  {'date': datetime.datetime(2012, 6, 25, 0, 0),
   'price': Decimal128('9.4562')},
  {'date': datetime.datetime(2016, 11, 9, 0, 0),
   'price': Decimal128('70.3236')},
  {'date': datetime.datetime(2005, 8, 17, 0, 0),
   'price': Decimal128('68.4043')},
  {'date': datetime.datetime(2010, 12, 16, 0, 0),
   'price': Decimal128('13.4266')},
  {'da

#### Data Query

Query the transactions collection to return the average transaction value for a given property in a given period of time.

In [151]:
from datetime import datetime

# Define the start and end dates for the period of time you want to search
start_date = datetime.strptime("2008-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2022-12-31", "%Y-%m-%d")

match_stage11 = [{
    "$match": {
        "listing_id": listing_sample_id,
        "transactions.date": {
            "$gte": start_date,
            "$lte": end_date
        }
    }
}, {"$unwind": "$transactions"}]

group_stage11 = {
    "$group": {
        "_id": "$listing_id",
        "avg_transaction_value": {
            "$avg": "$transactions.price"
        }
    }
}

pipeline11 = [
    *match_stage11,
    group_stage11
]

# Execute the aggregation pipeline
result11 = db.transactions.aggregate(pipeline11)
avg_transaction_value = result11.next()["avg_transaction_value"]

print(
    f"Average transaction value for property {listing_sample_id} between {start_date} and {end_date} is: {avg_transaction_value}")

Average transaction value for property 10006546 between 2008-01-01 00:00:00 and 2022-12-31 00:00:00 is: 18.40265714285714285714285714285714


### Question 12

We wish to have a summary webpage that displays information about our top destinations. This webpage should display for each of the top 10 cities some basic information about our operations in the area (number of properties by type for example, average price by type) but you can choose the metrics. For each of the top 10 cities it should also provide some basic information about the top 3 properties in each city (price, number of review, whatever you think useful) to show an example of the properties available in the area. We would like to keep this webpage up to date as information changes.

#### Top 10 Cities

Query the listings collection to return the top 10 cities by number of listings

In [152]:
# Group by market and calculate the average price, number of reviews, number of amenities, average review score rating and average review score value
group_stage12 = {
    "$group": {
        "_id": "$address.market",
        "total_listings": {"$sum": 1},
        "avg_reviews": {"$avg": "$number_of_reviews"},
        "avg_amenities": {"$avg": {"$size": "$amenities"}},
        "avg_price": {"$avg": "$price"},
        "avg_score_rating": {"$avg": "$REVIEWS_SCORES.rating"},
        "avg_score_value": {"$avg": "$REVIEWS_SCORES.value"},
    }
}

# Sort by total listings in desc order, reviews in desc order, amenities in desc order, price in asc order, score rating in desc order, score value in desc order
sort_stage12 = {
    "$sort": {
        "total_listings": -1,
        "avg_reviews": -1,
        "avg_amenities": -1,
        "avg_price": 1,  # Sort by avg price in asc order
        "avg_score_rating": -1,
        "avg_score_value": -1,
    }
}

limit_stage12 = {
    "$limit": 10
}

pipeline12 = [
    group_stage12,
    sort_stage12,
    limit_stage12,
]

# Execute the aggregation pipeline
result12 = db.listings.aggregate(pipeline12)
top_cities = list(result12)
top_cities

[{'_id': 'Istanbul',
  'total_listings': 660,
  'avg_reviews': 8.598484848484848,
  'avg_amenities': 18.545454545454547,
  'avg_price': Decimal128('367.9454545454545454545454545454545'),
  'avg_score_rating': 91.59415584415585,
  'avg_score_value': 9.239482200647249},
 {'_id': 'Montreal',
  'total_listings': 648,
  'avg_reviews': 17.192901234567902,
  'avg_amenities': 22.067901234567902,
  'avg_price': Decimal128('100.2330246913580246913580246913580'),
  'avg_score_rating': 92.89452332657201,
  'avg_score_value': 9.40040650406504},
 {'_id': 'Barcelona',
  'total_listings': 632,
  'avg_reviews': 30.544303797468356,
  'avg_amenities': 21.264240506329113,
  'avg_price': Decimal128('100.9477848101265822784810126582278'),
  'avg_score_rating': 90.9113924050633,
  'avg_score_value': 8.989429175475687},
 {'_id': 'Hong Kong',
  'total_listings': 619,
  'avg_reviews': 21.32956381260097,
  'avg_amenities': 17.22778675282714,
  'avg_price': Decimal128('762.4781906300484652665589660743134'),
  'av

In [153]:
top_cities_names = [city["_id"] for city in top_cities]
print(top_cities_names)

['Istanbul', 'Montreal', 'Barcelona', 'Hong Kong', 'Sydney', 'New York', 'Rio De Janeiro', 'Porto', 'Oahu', 'Maui']


#### Properties Types Details

In [154]:
# Add a $match stage to filter by cities
match_stage12_1 = {
    "$match": {
        "address.market": {"$in": top_cities_names}  # Filter by cities in the predefined list
    }
}

# Group by property type and calculate the average price and number of properties
group_stage12_1 = {
    "$group": {
        "_id": {"city": "$address.market", "property_type": "$property_type"},
        "count": {"$sum": 1},
        "avg_price": {"$avg": "$price"}
    }
}

project_stage12_1 = {
    "$project": {
        "_id": 0,
        "city": "$_id.city",
        "property_type": "$_id.property_type",
        "count": 1,
        "avg_price": 1
    }
}

sort_stage12_1 = {
    "$sort": {
        "city": 1,
        "property_type": 1,
    }
}

pipeline12_1 = [
    match_stage12_1,
    group_stage12_1,
    project_stage12_1,
    sort_stage12_1,
]

result12_1 = list(db.listings.aggregate(pipeline12_1))
result12_1

[{'count': 3,
  'avg_price': Decimal128('189.3333333333333333333333333333333'),
  'city': 'Barcelona',
  'property_type': 'Aparthotel'},
 {'count': 534,
  'avg_price': Decimal128('91.84644194756554307116104868913858'),
  'city': 'Barcelona',
  'property_type': 'Apartment'},
 {'count': 8,
  'avg_price': Decimal128('37.75'),
  'city': 'Barcelona',
  'property_type': 'Bed and breakfast'},
 {'count': 1,
  'avg_price': Decimal128('35.00'),
  'city': 'Barcelona',
  'property_type': 'Boat'},
 {'count': 3,
  'avg_price': Decimal128('109.3333333333333333333333333333333'),
  'city': 'Barcelona',
  'property_type': 'Boutique hotel'},
 {'count': 3,
  'avg_price': Decimal128('1051.00'),
  'city': 'Barcelona',
  'property_type': 'Casa particular (Cuba)'},
 {'count': 15,
  'avg_price': Decimal128('140.7333333333333333333333333333333'),
  'city': 'Barcelona',
  'property_type': 'Condominium'},
 {'count': 1,
  'avg_price': Decimal128('100.00'),
  'city': 'Barcelona',
  'property_type': 'Farm stay'},
 {

#### Top 3 Properties

In [155]:
# Add a $match stage to filter by cities
match_stage12_2 = {
    "$match": {
        "address.market": {"$in": top_cities_names}
    }
}

group_stage_12_2 = {
    "$group": {
        "_id": "$address.market",
        "total_listings": {"$sum": 1},
        "top_listings": {
            "$push": {
                "id": "$_id",
                "name": "$name",
                "listing_url": "$listing_url",
                "accommodates": "$accommodates",
                "amenities": "$amenities",
                "score_rating": "$REVIEWS_SCORES.rating",
                "property_type": "$property_type",
                "price": "$price",
                "number_of_reviews": "$number_of_reviews"
            }
        }
    }
}

# Add a $sort stage to sort the top listings array by price in descending order
sort_stage12_2 = {
    "$sort": {
        "top_listings.score_rating": -1
    }
}

# Add a $project stage to limit the top listings array to only the top 3 listings
project_stage12_2 = {
    "$project": {
        "_id": 1,
        "total_listings": 1,
        "top_listings": {"$slice": ["$top_listings", 3]}
    }
}

pipeline12_2 = [
    match_stage12_2,
    group_stage_12_2,
    sort_stage12_2,
    project_stage12_2
]

result12_2 = list(db.listings.aggregate(pipeline12_2))
result12_2

[{'_id': 'Istanbul',
  'total_listings': 660,
  'top_listings': [{'id': '21654414',
    'name': 'Boutique Room in Taksim 5 Free wi_fi',
    'listing_url': 'https://www.airbnb.com/rooms/21654414',
    'accommodates': 2,
    'amenities': ['TV',
     'Wifi',
     'Air conditioning',
     'Paid parking off premises',
     'Heating',
     'Smoke detector',
     'Fire extinguisher',
     'Essentials',
     'Shampoo',
     'Lock on bedroom door',
     'Hangers',
     'Hair dryer',
     'Iron',
     'Laptop friendly workspace',
     'Hot water',
     'Luggage dropoff allowed',
     'Long term stays allowed',
     'Host greets you'],
    'score_rating': 86,
    'property_type': 'Hotel',
    'price': Decimal128('158.00'),
    'number_of_reviews': 7},
   {'id': '29354797',
    'name': 'Beşiktaş Central 1+1 Fully Fresh New Furnished',
    'listing_url': 'https://www.airbnb.com/rooms/29354797',
    'accommodates': 3,
    'amenities': ['TV',
     'Cable TV',
     'Wifi',
     'Kitchen',
     'Paid p

### Question 13

Add a new property, this property should have a new host, and should be located in one of the top 10 cities. The host selects the top 10 most common amenities to be listed for the property.

#### Add a new host

In [156]:
import random

# Randomly select a city from the top 10 cities
city = random.choice(top_cities_names)
top_amenities = top_amenities[:10]

In [157]:
db.hosts.delete_many({"_id": "999999999"})
new_host = {
    "_id": "999999999",
    "schema_version": 2,
    "host_about": "This is a test host!!!",
    "host_has_profile_pic": True,
    "host_identity_verified": True,
    "host_is_superhost": False,
    "host_listings_count": 1,
    "host_location": city,
    "host_name": "New Host",
    "host_neighbourhood": "",
    "host_picture_url": "https://example.com/profile.jpg",
    "host_response_rate": 100,
    "host_response_time": "within an hour",
    "host_thumbnail_url": "https://example.com/profile_thumbnail.jpg",
    "host_total_listings_count": 1,
    "host_url": "https://www.airbnb.com/users/show/12345678",
    "host_verifications": [
        "email",
        "phone",
        "reviews",
        "jumio",
        "offline_government_id",
        "government_id"
    ]
}

# Insert the new host document into the database
result13 = db.hosts.insert_one(new_host)
new_host_id = result13.inserted_id
print("Inserted document id:", new_host_id)

Inserted document id: 999999999


#### Add a new property

Get the full address of the randomly selected city from the top 10 cities and use it for the new property.

In [158]:
full_address_of_topCity = db.listings.find_one({"address.market": city}, {"address": 1})
full_address_of_topCity['address']

{'street': 'Barcelona, Catalunya, Spain',
 'suburb': "L'Antiga Esquerra de l'Eixample",
 'government_area': "l'Antiga Esquerra de l'Eixample",
 'market': 'Barcelona',
 'country': 'Spain',
 'country_code': 'ES',
 'location': {'type': 'Point',
  'coordinates': [2.15268, 41.39029],
  'is_location_exact': False}}

Insert the new property document into the database.

In [171]:
db.listings.delete_many({"_id": "1010101010"})
new_property = {
    "_id": "1010101011",
    "schema_version": 2,
    "address": full_address_of_topCity['address'],
    "amenities": top_amenities,
    "bathrooms": 1,
    "bedrooms": 1,
    "beds": 1,
    "host_id": new_host_id,
    "host_url": "https://www.airbnb.com/users/show/12345678",
    "listing_url": "https://www.airbnb.com/rooms/999999999",
    "name": "Test Property",
    "number_of_reviews": 0,
    "price": 100,
    "property_type": "Apartment",
    "space": "This is a test property!!!",
    "summary": "This is a test property!!!",
    "REVIEWS_SCORES": {
        "cleanliness": 0,
        "checkin": 0,
        "communication": 0,
        "location": 0,
        "value": 0,
        "rating": 0
    },
}

# Insert the new property document into the database
result13_1 = db.listings.insert_one(new_property)
new_property_id = result13_1.inserted_id
db.listings.find_one({"_id": new_property_id})

{'_id': '1010101011',
 'schema_version': 2,
 'address': {'street': 'Barcelona, Catalunya, Spain',
  'suburb': "L'Antiga Esquerra de l'Eixample",
  'government_area': "l'Antiga Esquerra de l'Eixample",
  'market': 'Barcelona',
  'country': 'Spain',
  'country_code': 'ES',
  'location': {'type': 'Point',
   'coordinates': [2.15268, 41.39029],
   'is_location_exact': False}},
 'amenities': ['Bed linens',
  'Heated towel rack',
  'Sauna',
  'Host greets you',
  'Bathtub',
  'Sun loungers',
  'Pack ’n Play/travel crib',
  'Wifi',
  'Children’s dinnerware',
  'Garden or backyard'],
 'bathrooms': 1,
 'bedrooms': 1,
 'beds': 1,
 'host_id': '999999999',
 'host_url': 'https://www.airbnb.com/users/show/12345678',
 'listing_url': 'https://www.airbnb.com/rooms/999999999',
 'name': 'Test Property',
 'number_of_reviews': 0,
 'price': 100,
 'property_type': 'Apartment',
 'space': 'This is a test property!!!',
 'summary': 'This is a test property!!!',
 'REVIEWS_SCORES': {'cleanliness': 0,
  'checkin': 

### Question 14

Add a new review to this property, the review should be from one of our top 20 reviewers.

#### Top 20 reviewers

In [160]:
# Define the aggregation pipeline
pipeline = [
    {"$group": {"_id": {"reviewer_id": "$reviewer_id", "reviewer_name": "$reviewer_name"}, "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 20}
]

# Execute the aggregation query
result = db.reviews.aggregate(pipeline)
top_reviewers = list(result)
top_reviewers

[{'_id': {'reviewer_id': '20775242', 'reviewer_name': 'Filipe'}, 'count': 24},
 {'_id': {'reviewer_id': '67084875', 'reviewer_name': 'Nick'}, 'count': 13},
 {'_id': {'reviewer_id': '2961855', 'reviewer_name': 'Uge'}, 'count': 10},
 {'_id': {'reviewer_id': '20991911', 'reviewer_name': 'Lisa'}, 'count': 9},
 {'_id': {'reviewer_id': '162027327', 'reviewer_name': 'Thien'}, 'count': 9},
 {'_id': {'reviewer_id': '12679057', 'reviewer_name': 'Jodi'}, 'count': 8},
 {'_id': {'reviewer_id': '1705870', 'reviewer_name': 'David'}, 'count': 8},
 {'_id': {'reviewer_id': '55241576', 'reviewer_name': 'Courtney'}, 'count': 8},
 {'_id': {'reviewer_id': '60816198', 'reviewer_name': 'Todd'}, 'count': 8},
 {'_id': {'reviewer_id': '69140895', 'reviewer_name': 'Lisa'}, 'count': 8},
 {'_id': {'reviewer_id': '78093968', 'reviewer_name': 'David'}, 'count': 7},
 {'_id': {'reviewer_id': '47303133', 'reviewer_name': 'Lance'}, 'count': 7},
 {'_id': {'reviewer_id': '25715809', 'reviewer_name': 'Megan'}, 'count': 6},


In [161]:
# Get reviewers details
top_reviewers_details = [reviewer['_id'] for reviewer in top_reviewers]
top_reviewers_details

[{'reviewer_id': '20775242', 'reviewer_name': 'Filipe'},
 {'reviewer_id': '67084875', 'reviewer_name': 'Nick'},
 {'reviewer_id': '2961855', 'reviewer_name': 'Uge'},
 {'reviewer_id': '20991911', 'reviewer_name': 'Lisa'},
 {'reviewer_id': '162027327', 'reviewer_name': 'Thien'},
 {'reviewer_id': '12679057', 'reviewer_name': 'Jodi'},
 {'reviewer_id': '1705870', 'reviewer_name': 'David'},
 {'reviewer_id': '55241576', 'reviewer_name': 'Courtney'},
 {'reviewer_id': '60816198', 'reviewer_name': 'Todd'},
 {'reviewer_id': '69140895', 'reviewer_name': 'Lisa'},
 {'reviewer_id': '78093968', 'reviewer_name': 'David'},
 {'reviewer_id': '47303133', 'reviewer_name': 'Lance'},
 {'reviewer_id': '25715809', 'reviewer_name': 'Megan'},
 {'reviewer_id': '76782210', 'reviewer_name': 'Assis'},
 {'reviewer_id': '93859831', 'reviewer_name': 'Pierre'},
 {'reviewer_id': '61469899', 'reviewer_name': 'Erik'},
 {'reviewer_id': '73708321', 'reviewer_name': 'Gonzalo'},
 {'reviewer_id': '128210181', 'reviewer_name': 'Br

#### Random Reviewer

In [162]:
# Randomly select a reviewer from the top 20 reviewers
reviewer = random.choice(top_reviewers_details)
reviewer

{'reviewer_id': '2961855', 'reviewer_name': 'Uge'}

#### Add new review

In [168]:
new_review = {
    "_id": "111111118",
    "comments": "This is a test review!!!",
    "date": datetime.now(),
    "listing_id": new_property_id,
    "reviewer_id": reviewer['reviewer_id'],
    "reviewer_name": reviewer['reviewer_name']
}

# Insert the new review document into the database
result14 = db.reviews.insert_one(new_review)
new_review_id = result14.inserted_id
print("Inserted review document id:", new_review_id)

Inserted review document id: 111111118


#### Update Computed Fields

Update the computed fields here number of reviews and latest reviews array for the new review document just added.

In [169]:
# Update the listing collection with the new review data
recent_reviews = list(db.reviews.find({"listing_id": new_property_id}).sort("date", -1).limit(5))
db.listings.update_one(
    {"_id": new_property_id},
    {
        "$inc": {"number_of_reviews": 1},
        "$push": {"LATEST_REVIEWS": {"$each": recent_reviews, "$slice": -5}},
    }
)

listing_data = db.listings.find_one({"_id": new_property_id}, {"number_of_reviews": 1, "LATEST_REVIEWS": 1})
listing_data

{'_id': '1010101010',
 'number_of_reviews': 2,
 'LATEST_REVIEWS': [{'_id': '111111117',
   'comments': 'This is a test review!!!',
   'date': datetime.datetime(2023, 4, 23, 11, 45, 48, 471000),
   'listing_id': '1010101010',
   'reviewer_id': '2961855',
   'reviewer_name': 'Uge'},
  {'_id': '111111118',
   'comments': 'This is a test review!!!',
   'date': datetime.datetime(2023, 4, 23, 11, 47, 35, 433000),
   'listing_id': '1010101010',
   'reviewer_id': '2961855',
   'reviewer_name': 'Uge'},
  {'_id': '111111117',
   'comments': 'This is a test review!!!',
   'date': datetime.datetime(2023, 4, 23, 11, 45, 48, 471000),
   'listing_id': '1010101010',
   'reviewer_id': '2961855',
   'reviewer_name': 'Uge'}]}

### Question 15

Add a new review metric called x_factor to the new document and give a score of 10. Show that the average across all metrics is correctly calculated for this listing, with the query you previously developed.

In [178]:
db.listings.update_one(
    {"_id": new_property_id},
    {
        "$set": {"REVIEWS_SCORES.x_factor": 10},
        "$inc": {"REVIEWS_SCORES.rating": 1}
    },
)

## x_factor = (old_average * (ratings_count - 1) + new_rating) / ratings_count

db.listings.find_one({"_id": new_property_id}, {"reviews_scores_rating": 1, "REVIEWS_SCORES": 1})

{'_id': '1010101011',
 'REVIEWS_SCORES': {'cleanliness': 0,
  'checkin': 0,
  'communication': 0,
  'location': 0,
  'value': 2.0,
  'rating': 2,
  'x_factor': 10}}

In [179]:
db.listings.update_one(
    {"_id": new_property_id}, [
        {
            "$set": {"REVIEWS_SCORES.value": {
                "$avg": [
                    "$REVIEWS_SCORES.cleanliness",
                    "$REVIEWS_SCORES.checkin",
                    "$REVIEWS_SCORES.communication",
                    "$REVIEWS_SCORES.location",
                    "$REVIEWS_SCORES.x_factor",
                ]
            }},
        }
    ]
)

## rating_count ++1
## x_factor = (old_average * (ratings_count - 1) + new_rating) / ratings_count
## value = (old_average * (ratings_count - 1) + (total of other ratings)) / ratings_count

In [180]:
db.listings.find_one({"_id": new_property_id}, {"reviews_scores_rating": 1, "REVIEWS_SCORES": 1})

{'_id': '1010101011',
 'REVIEWS_SCORES': {'cleanliness': 0,
  'checkin': 0,
  'communication': 0,
  'location': 0,
  'value': 2.0,
  'rating': 2,
  'x_factor': 10}}

#### Clarification

Actually I can not understand the business logic here for that rating scores part,

Is each reviews added with a rating and the average of scores is calculated based on the number of ratings?
What the relation between theses different scores and the rating score & rating value ?
How can I calculate the average of that scores if I don't know the number of ratings for each score?
I'm so confused here, please help me to understand this part.